# EUR/TND Forex Options Analysis

This notebook provides an interactive analysis of forex options on the EUR/TND currency pair using the pricing system.

## Setup

First, let's import the necessary modules and initialize the system.

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from IPython.display import display

# Add the src directory to the path
sys.path.insert(0, os.path.abspath('..'))

# Import forex options modules
from src.forex_options.market_data import MarketDataGenerator
from src.forex_options.options_gen import OptionsGenerator
from src.forex_options.pricing import BlackScholesFX, MertonJumpDiffusion, SABR, PricingEngine
from src.forex_options.portfolio import PortfolioManager
from src.forex_options.evaluation import ModelEvaluator
from src.forex_options.visualization import VisualizationManager

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.2)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.precision', 4)

## Market Data

Let's initialize and load (or generate) the market data.

In [ ]:
# Initialize market data
market_data = MarketDataGenerator(
    start_date='2023-01-01',
    end_date='2024-12-31',
    base_eur_tnd_rate=3.35,
    eur_rate_mean=0.03,
    tnd_rate_mean=0.08
)

# Try to load data, if not available, generate it
data_dir = '../data'
if not market_data.check_or_generate_data(data_dir):
    print("Failed to load or generate market data.")

### Explore Exchange Rate Data

In [ ]:
# Display exchange rate data
print("Exchange Rate Summary:")
display(market_data.eur_tnd_daily.describe())

# Plot exchange rate
plt.figure(figsize=(12, 6))
plt.plot(market_data.eur_tnd_daily['Date'], market_data.eur_tnd_daily['EUR/TND'])
plt.title('EUR/TND Exchange Rate')
plt.xlabel('Date')
plt.ylabel('Rate')
plt.grid(True)
plt.show()

# Plot volatility
plt.figure(figsize=(12, 6))
for window in [5, 21, 63]:
    vol_col = f'{window}d_Volatility'
    plt.plot(market_data.eur_tnd_daily['Date'], market_data.eur_tnd_daily[vol_col], label=f'{window}-day volatility')
plt.title('EUR/TND Realized Volatility')
plt.xlabel('Date')
plt.ylabel('Annualized Volatility')
plt.legend()
plt.grid(True)
plt.show()

### Explore Interest Rate Data

In [ ]:
# Display interest rate data
print("EUR Interest Rate Summary:")
display(market_data.eur_rates_monthly.describe())

print("TND Interest Rate Summary:")
display(market_data.tnd_rates_monthly.describe())

# Plot interest rates
plt.figure(figsize=(12, 6))
plt.plot(market_data.eur_rates_monthly['Date'], market_data.eur_rates_monthly['EUR_Rate'], label='EUR Interest Rate')
plt.plot(market_data.tnd_rates_monthly['Date'], market_data.tnd_rates_monthly['TND_Rate'], label='TND Interest Rate')
plt.title('Interest Rates')
plt.xlabel('Date')
plt.ylabel('Rate')
plt.legend()
plt.grid(True)
plt.show()

# Calculate interest rate differential
dates = pd.merge(market_data.eur_rates_monthly[['Date']], market_data.tnd_rates_monthly[['Date']], on='Date')['Date']
eur_rates = market_data.eur_rates_monthly[market_data.eur_rates_monthly['Date'].isin(dates)]
tnd_rates = market_data.tnd_rates_monthly[market_data.tnd_rates_monthly['Date'].isin(dates)]
differential = tnd_rates['TND_Rate'].values - eur_rates['EUR_Rate'].values

plt.figure(figsize=(12, 6))
plt.plot(dates, differential)
plt.title('TND-EUR Interest Rate Differential')
plt.xlabel('Date')
plt.ylabel('Rate Differential')
plt.grid(True)
plt.show()

## Options Portfolio

Now, let's initialize and load (or generate) the options portfolio.

In [ ]:
# Initialize options generator
options_generator = OptionsGenerator(
    market_data,
    simulation_year=2024,
    max_notional=10_000_000
)

# Try to load portfolio, if not available, generate it
if not options_generator.check_or_generate_options(data_dir):
    print("Failed to load or generate options portfolio.")

### Explore Options Portfolio

In [ ]:
# Display options portfolio summary
portfolio = options_generator.options_portfolio

print(f"Number of options: {len(portfolio)}")
print(f"Total notional: €{portfolio['NotionalEUR'].sum():,.2f}")
print(f"Average tenor: {portfolio['Tenor'].mean():.1f} days")
print(f"Average moneyness: {portfolio['Moneyness'].mean():.4f}")

# Display a few sample options
display(portfolio.head())

# Plot maturity distribution
plt.figure(figsize=(10, 6))
sns.histplot(portfolio['Tenor'], bins=20)
plt.title('Distribution of Option Maturities')
plt.xlabel('Tenor (days)')
plt.ylabel('Count')
plt.grid(True)
plt.show()

# Plot moneyness distribution
plt.figure(figsize=(10, 6))
sns.histplot(portfolio['Moneyness'], bins=20)
plt.title('Distribution of Option Moneyness')
plt.xlabel('Moneyness (Strike/Spot)')
plt.ylabel('Count')
plt.axvline(x=1, color='red', linestyle='--', label='At-the-money')
plt.legend()
plt.grid(True)
plt.show()

# Plot issue dates
plt.figure(figsize=(10, 6))
sns.histplot(portfolio['IssueDate'], bins=12)
plt.title('Distribution of Option Issue Dates')
plt.xlabel('Issue Date')
plt.ylabel('Count')
plt.grid(True)
plt.show()

## Option Pricing Models

Let's explore the different pricing models by pricing a sample option.

In [ ]:
# Get a sample option
sample_option = portfolio.iloc[0].to_dict()
display(pd.DataFrame([sample_option]))

# Get market data for pricing date
pricing_date = pd.Timestamp('2024-06-01')
market_info = market_data.get_market_data(pricing_date)

# Extract parameters
S = market_info['spot_rate']  # Spot rate
K = sample_option['StrikePrice']  # Strike
issue_date = sample_option['IssueDate']
maturity_date = sample_option['MaturityDate']
T = (maturity_date - pricing_date).days / 365.0  # Time to maturity in years
r_d = market_info['eur_rate']  # EUR rate
r_f = market_info['tnd_rate']  # TND rate
sigma = market_info['volatility']  # Volatility

print(f"Pricing parameters:")
print(f"Spot rate (S): {S:.4f}")
print(f"Strike price (K): {K:.4f}")
print(f"Time to maturity (T): {T:.4f} years ({(maturity_date - pricing_date).days} days)")
print(f"EUR interest rate (r_d): {r_d:.2%}")
print(f"TND interest rate (r_f): {r_f:.2%}")
print(f"Volatility (sigma): {sigma:.2%}")
print(f"Moneyness (K/S): {K/S:.4f}")

### Black-Scholes (Garman-Kohlhagen) Model

In [ ]:
# Price with Black-Scholes
bs_result = BlackScholesFX.price(S, K, T, r_d, r_f, sigma, 'call')

print(f"Black-Scholes Price: {bs_result.price:.4f}")
print(f"Delta: {bs_result.delta:.4f}")
print(f"Gamma: {bs_result.gamma:.6f}")
print(f"Vega: {bs_result.vega:.6f}")
print(f"Theta: {bs_result.theta:.6f}")
print(f"Rho (domestic): {bs_result.rho_d:.6f}")
print(f"Rho (foreign): {bs_result.rho_f:.6f}")

### Merton Jump Diffusion Model

In [ ]:
# Price with Merton Jump Diffusion
# Define jump parameters
lam = 1.0      # Jump frequency (1 jump per year on average)
mu_j = -0.05   # Average jump size (-5%)
sigma_j = 0.08 # Jump size volatility (8%)

mjd_result = MertonJumpDiffusion.price(S, K, T, r_d, r_f, sigma, lam, mu_j, sigma_j, 'call')

print(f"Merton Jump Diffusion Price: {mjd_result.price:.4f}")
print(f"Delta: {mjd_result.delta:.4f}")
print(f"Gamma: {mjd_result.gamma:.6f}")
print(f"Vega: {mjd_result.vega:.6f}")
print(f"Theta: {mjd_result.theta:.6f}")
print(f"Rho (domestic): {mjd_result.rho_d:.6f}")
print(f"Rho (foreign): {mjd_result.rho_f:.6f}")

# Compare with Black-Scholes
print(f"\nDifference from Black-Scholes:")
print(f"Price difference: {mjd_result.price - bs_result.price:.4f} ({(mjd_result.price / bs_result.price - 1) * 100:.2f}%)")
print(f"Delta difference: {mjd_result.delta - bs_result.delta:.4f}")

### SABR Stochastic Volatility Model

In [ ]:
# Price with SABR
# Define SABR parameters
alpha = sigma   # Initial volatility 
beta = 0.5      # CEV parameter (0=normal, 1=lognormal)
rho = -0.3      # Correlation between price and volatility
nu = 0.4        # Volatility of volatility

sabr_result = SABR.price(S, K, T, r_d, r_f, alpha, beta, rho, nu, 'call')

print(f"SABR Price: {sabr_result.price:.4f}")
print(f"Delta: {sabr_result.delta:.4f}")
print(f"Gamma: {sabr_result.gamma:.6f}")
print(f"Vega: {sabr_result.vega:.6f}")
print(f"Theta: {sabr_result.theta:.6f}")
print(f"Rho (domestic): {sabr_result.rho_d:.6f}")
print(f"Rho (foreign): {sabr_result.rho_f:.6f}")

# Compare with Black-Scholes
print(f"\nDifference from Black-Scholes:")
print(f"Price difference: {sabr_result.price - bs_result.price:.4f} ({(sabr_result.price / bs_result.price - 1) * 100:.2f}%)")
print(f"Delta difference: {sabr_result.delta - bs_result.delta:.4f}")

# Compare with Merton Jump Diffusion
print(f"\nDifference from Merton Jump Diffusion:")
print(f"Price difference: {sabr_result.price - mjd_result.price:.4f} ({(sabr_result.price / mjd_result.price - 1) * 100:.2f}%)")
print(f"Delta difference: {sabr_result.delta - mjd_result.delta:.4f}")

## SABR Volatility Surface

Let's visualize the SABR volatility surface to understand the implied volatility patterns.

In [ ]:
# Parameters for the surface
strikes = np.linspace(0.8 * S, 1.2 * S, 20)
tenors = np.linspace(0.1, 1.0, 10)

# Create mesh grid
K_mesh, T_mesh = np.meshgrid(strikes, tenors)
implied_vols = np.zeros_like(K_mesh)

# Calculate implied volatilities
for i in range(tenors.shape[0]):
    for j in range(strikes.shape[0]):
        # Forward price
        F = S * np.exp((r_d - r_f) * tenors[i])
        
        # SABR implied volatility
        implied_vols[i, j] = SABR.implied_vol(F, strikes[j], tenors[i], alpha, beta, rho, nu)

# Plot the surface
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_surface(K_mesh, T_mesh, implied_vols, cmap='viridis', edgecolor='none')

ax.set_xlabel('Strike')
ax.set_ylabel('Tenor (years)')
ax.set_zlabel('Implied Volatility')
ax.set_title('SABR Implied Volatility Surface')

fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)
plt.show()

# Plot volatility smiles for different tenors
plt.figure(figsize=(12, 6))

for i, tenor_idx in enumerate([0, 2, 5, 9]):
    if tenor_idx < len(tenors):
        plt.plot(strikes, implied_vols[tenor_idx, :], label=f'T = {tenors[tenor_idx]:.2f}')

plt.axvline(x=S, color='black', linestyle='--', label='Spot')

plt.title('SABR Volatility Smiles for Different Tenors')
plt.xlabel('Strike')
plt.ylabel('Implied Volatility')
plt.legend()
plt.grid(True)
plt.show()

## Portfolio Pricing and Risk Analysis

Now, let's price the entire portfolio and analyze the risks.

In [ ]:
# Initialize portfolio manager
portfolio_manager = PortfolioManager(market_data, options_generator)

# Define models and parameters
models = ['black_scholes', 'merton_jump', 'sabr']
model_params = {
    'merton_jump': {
        'lambda': 1.0,
        'mu_j': -0.05,
        'sigma_j': 0.08
    },
    'sabr': {
        'beta': 0.5,
        'rho': -0.3,
        'nu': 0.4
    }
}

# Price portfolio
pricing_date = pd.Timestamp('2024-06-01')
results = portfolio_manager.price_portfolio(pricing_date, models, model_params)

# Display results summary
print(f"Portfolio pricing completed with {len(results)} results across {len(models)} models.")
model_summary = results.groupby('model').agg(
    count=('option_id', 'count'),
    total_value=('option_value_eur', 'sum'),
    avg_price=('price', 'mean'),
    avg_delta=('delta', 'mean')
).reset_index()

display(model_summary)

### Portfolio Risk Analysis

In [ ]:
# Calculate portfolio risk
portfolio_risk = portfolio_manager.calculate_portfolio_risk()

# Display risk metrics
for model, risk in portfolio_risk.items():
    print(f"\n{model.upper()} Model:")
    print(f"Total value: €{risk.total_value_eur:,.2f}")
    print(f"Total delta: {risk.total_delta:,.2f}")
    if risk.total_gamma is not None:
        print(f"Total gamma: {risk.total_gamma:,.6f}")
    if risk.total_vega is not None:
        print(f"Total vega: {risk.total_vega:,.6f}")
    if risk.total_theta is not None:
        print(f"Total theta: {risk.total_theta:,.6f}")
    print(f"Number of options: {risk.count}")

# Plot risk metrics comparison
metrics = ['total_value_eur', 'total_delta']
metric_names = {'total_value_eur': 'Total Value (EUR)', 'total_delta': 'Total Delta'}

for metric in metrics:
    plt.figure(figsize=(10, 6))
    values = [getattr(risk, metric) for risk in portfolio_risk.values()]
    plt.bar(list(portfolio_risk.keys()), values)
    plt.title(f'Portfolio {metric_names[metric]} by Model')
    plt.xlabel('Model')
    plt.ylabel(metric_names[metric])
    plt.grid(True, axis='y')
    plt.show()

### Portfolio Exposure Analysis

In [ ]:
# Calculate exposure by maturity
exp_maturity = portfolio_manager.calculate_exposure_by_maturity()

# Convert to DataFrame for easier plotting
exp_mat_df = pd.DataFrame(exp_maturity).T

# Display exposure by maturity
display(exp_mat_df)

# Stacked bar chart
ax = exp_mat_df.plot(kind='bar', stacked=True, figsize=(12, 6))
ax.set_title('Portfolio Exposure by Maturity')
ax.set_xlabel('Model')
ax.set_ylabel('Exposure (EUR)')
plt.grid(True, axis='y')
plt.show()

# Calculate exposure by moneyness
exp_moneyness = portfolio_manager.calculate_exposure_by_moneyness()

# Convert to DataFrame for easier plotting
exp_mon_df = pd.DataFrame(exp_moneyness).T

# Display exposure by moneyness
display(exp_mon_df)

# Stacked bar chart
ax = exp_mon_df.plot(kind='bar', stacked=True, figsize=(12, 6))
ax.set_title('Portfolio Exposure by Moneyness')
ax.set_xlabel('Model')
ax.set_ylabel('Exposure (EUR)')
plt.grid(True, axis='y')
plt.show()

### Model Comparison

In [ ]:
# Calculate model comparison
comparison = portfolio_manager.calculate_model_comparison(reference_model='black_scholes')

# Display comparison
display(comparison)

# Plot price differences
plt.figure(figsize=(10, 6))
metrics = ['mean_price_diff_eur', 'median_price_diff_eur', 'max_price_diff_eur']
metric_labels = ['Mean Diff', 'Median Diff', 'Max Diff']

x = np.arange(len(comparison))
width = 0.25

for i, (metric, label) in enumerate(zip(metrics, metric_labels)):
    plt.bar(x + i*width, comparison[metric], width, label=label)

plt.xlabel('Model')
plt.ylabel('Price Difference (EUR)')
plt.title('Price Differences vs. Black-Scholes')
plt.xticks(x + width, comparison['model'])
plt.legend()
plt.grid(True, axis='y')
plt.show()

# Plot percentage differences
plt.figure(figsize=(10, 6))
plt.bar(comparison['model'], comparison['mean_price_diff_pct'])
plt.xlabel('Model')
plt.ylabel('Mean Price Difference (%)')
plt.title('Mean Price Difference vs. Black-Scholes (%)')
plt.grid(True, axis='y')
plt.show()

## Conclusion

This notebook has demonstrated the use of the forex options pricing system for EUR/TND. We've seen how different pricing models produce different results, and how the SABR model in particular can capture the volatility smile/skew observed in the market.

Key takeaways:
- The SABR model provides a more flexible framework for handling implied volatility patterns
- The Merton Jump Diffusion model helps account for sudden market movements
- Portfolio exposure analysis helps understand risk concentrations
- Model comparison highlights the differences between pricing approaches

For production use, it would be important to calibrate the models to actual market prices of liquid options, and to regularly update the calibration as market conditions change.